In [ ]:
import os, re
import pickle
import numpy as np
import pandas as pd
import random

#matplotlib
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.cm import ScalarMappable
font = {'family' : 'serif',
        'style' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)

import matplotlib.colors as mc
from mpl_toolkits.axes_grid1 import make_axes_locatable

#scipy
from scipy import ndimage
from scipy.stats import gaussian_kde

import rioxarray
import xarray
import rasterio
import xarray as xr

## GDAL
from osgeo import gdal, osr, ogr

import utils
import CS_method

In [ ]:
region = 'CalaMillor'
year = '2018'
data_path = '***'

## Bathymetry

In [ ]:
## In-situ bathymetry
file = '{}_Insitu_bathymetry_EPSG32631_{}.tif'.format(region, year)
bathymetry, BBox, crs = utils.load_geotiff(os.path.join(data_path, 'InSitu', file))
mx = np.ma.masked_invalid(bathymetry)

## transform coordinates
new_ul = utils.transform_coordinates(BBox[0], inputEPSG=crs, outputEPSG=4326)
new_lr = utils.transform_coordinates(BBox[1], inputEPSG=crs, outputEPSG=4326)

# ROI
roi_coord = {'N': new_ul[0], 'W': new_ul[1], 'S': new_lr[0], 'E': new_lr[1]}

## Temporal Composite

In [ ]:
tiles = ['S2B_MSIL1C_20180917T103019_N0206_R108_T31SED_20180917T161011',
         'S2B_MSIL1C_20180920T104019_N0206_R008_T31SED_20180920T143638',
         'S2B_MSIL1C_20180930T104019_N0206_R008_T31SED_20180930T161005',
         'S2A_MSIL1C_20181002T103021_N0206_R108_T31SED_20181002T142247',
         'S2B_MSIL1C_20181017T103019_N0206_R108_T31SED_20181017T142233', 
         'S2A_MSIL1C_20181025T104121_N0206_R008_T31SED_20181025T125227']

In [ ]:
Zgr, Zr, Rs492, Rs559, Rs704, chl, coord = CS_method.composite_approach(tiles, roi_coord)

Zgr = ndimage.median_filter(Zgr, size=3)
Zgr[mx.mask] = np.nan

Zr = ndimage.median_filter(Zr, size=3)
Zr[mx.mask] = np.nan

Rs492[mx.mask] = np.nan
Rs559[mx.mask] = np.nan
Rs704[mx.mask] = np.nan
chl[mx.mask] = np.nan

## Outliers

In [ ]:
low_limit, upp_limit = utils.iq_outliers(Zgr)
Zgr[Zgr < low_limit] = np.nan
Zgr[Zgr > upp_limit] = np.nan

In [ ]:
low_limit, upp_limit = utils.iq_outliers(Zr)
Zr[Zr < low_limit] = np.nan
Zr[Zr > upp_limit] = np.nan

## Calibration

In [ ]:
n = random.sample(range(0, bathymetry.shape[0]), 12)
m = np.arange(0, bathymetry.shape[1], 3)

green_coefs = []
red_coefs = []
for i in n:

    Zgr_val = Zgr[i, m]
    Zr_val = Zr[i, m]
    depth_val = -bathymetry[i, m]

    #Zgreen
    mx = np.ma.masked_invalid(Zgr_val)
    Zgr_val = Zgr_val[~mx.mask]
    green_depth = depth_val[~mx.mask]

    ## linear regresion
    green_coefs.append(np.polyfit(Zgr_val, green_depth, 1))

    #Zred
    mx = np.ma.masked_invalid(Zr_val)
    Zr_val = Zr_val[~mx.mask]
    red_depth = depth_val[~mx.mask]

    Zr_val = Zr_val[red_depth < 5]
    red_depth = red_depth[red_depth < 5]

    ## linear regresion
    red_coefs.append(np.polyfit(Zr_val, red_depth, 1))

green_coefs = np.array(green_coefs)
max_idx = green_coefs[:,0].argmax()
green_coefs = np.delete(green_coefs, max_idx, 0)
min_idx = green_coefs[:,0].argmin()
green_coefs = np.delete(green_coefs, min_idx, 0)

red_coefs = np.array(red_coefs)
max_idx = red_coefs[:,0].argmax()
red_coefs = np.delete(red_coefs, max_idx, 0)
min_idx = red_coefs[:,0].argmin()
red_coefs = np.delete(red_coefs, min_idx, 0)
    
gr_coef = [np.median(green_coefs[:,0]), np.median(green_coefs[:,1])]
print(f'Green poly --> y = {gr_coef[0]:.4f}x + {gr_coef[1]:.4f}')
red_coef = [np.median(red_coefs[:,0]), np.median(red_coefs[:,1])]
print(f'Red poly --> y = {red_coef[0]:.4f}x + {red_coef[1]:.4f}')

In [ ]:
#Zgreen & Zred
poly_gr = np.poly1d(np.array(gr_coef))
poly_red = np.poly1d(red_coef)

## pSDBgreen & pSDBred
pSDBgreen = poly_gr(Zgr)
pSDBred = poly_red(Zr)

## Switching

In [ ]:
## Switching
a, b, Switching_SDB = CS_method.switching_model(pSDBgreen, pSDBred, Zgr_coef = 3.5, Zr_coef = 2)

## ODW

In [ ]:
## ODW
ODW_SDB = CS_method.odw_model(Switching_SDB, Rs492, Rs559, Rs704)
mx = np.ma.masked_invalid(ODW_SDB)

## Plot

In [ ]:
y_true = -bathymetry[~mx.mask]
y_pred = ODW_SDB[~mx.mask]
diff = y_pred - y_true

r2, MAE, MedAE, RMSE, MBE, MAPE = utils.scores(y_true, y_pred)
r2_sub10, MAE_sub10, MedAE_sub10, RMSE_sub10, MBE_sub10, MAPE_sub10 = utils.scores(y_true[y_true<10], y_pred[y_true<10])
r2_sup10, MAE_sup10, MedAE_sup10, RMSE_sup10, MBE_sup10, MAPE_sup10 = utils.scores(y_true[y_true>=10], y_pred[y_true>=10])

# Calculate the point density
x = y_true
y = y_pred
xy = np.vstack([x, y])
z = gaussian_kde(xy)(xy)

# Sort the points by density, so that the densest points are plotted last
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]

#Plot
fig, axs = plt.subplots(1, 2, figsize=(16, 6.5))
axs[0].scatter(x, y, c=z, s=10)
ax_ = axs[0].scatter(x, y, c=z, s=10)
plt.colorbar(ax_)
axs[0].plot([0,16], [0,16], 'k-', c='r', alpha=0.75, zorder=0)
axs[0].set_xlabel("In-situ Depth (m)")
axs[0].set_xlim(0,16)
axs[0].set_xticks(np.arange(.0, 18., 4.0))
axs[0].set_ylabel("SDB (m)")
axs[0].set_ylim(0,16)
axs[0].set_yticks(np.arange(.0, 18., 4.0))
axs[0].axis('square')

fig.text(.1, .86, f"$r^{2} = {r2:.2f}$")
fig.text(.1, .80, f"$RMSE = {RMSE:.2f}$")
fig.text(.1, .74, f"$N = {np.count_nonzero(~np.isnan(diff))}$")

axs[1].hist(diff[y_true<10], bins=80, alpha=0.5, label='$Depth<10m$', color='royalblue', density=True)
axs[1].set_xlim(-6,6)
axs[1].set_ylim(0, 1.5)
axs[1].get_yaxis().set_visible(False)
axs[1].set_xlabel("Residual error (m)")
axs[1].legend(loc=2, fontsize=20)
fig.text(.53, .77, f"$MBE={MBE_sub10:.2f}$")
fig.text(.53, .71, f"$MedAE={MedAE_sub10:.2f}$")
fig.text(.53, .65, f"$N={len(y_true[y_true<10])}$")

ax2 = axs[1].twinx()
ax2.hist(diff[y_true>=10], bins=35, alpha=0.6, label='$Depth \geq 10m$',color = 'darkorange', density=True)
ax2.set_ylabel("density")
ax2.set_ylim(0, 1.5)
ax2.legend(loc=1, fontsize=20)
fig.text(.735, .77, f"$MBE={MBE_sup10:.2f}$")
fig.text(.735, .71, f"$MedAE={MedAE_sup10:.2f}$")
fig.text(.735, .65, f"$N={len(y_true[y_true>=10])}$")
fig.text(.375, .96, f'{region} {year}')
plt.tight_layout()